In [1]:
import numpy as np
import cv2
 
cap = cv2.VideoCapture('002_RI_full.mp4')


fps = cap.get(cv2.CAP_PROP_FPS)

# 获取视频第一帧
ret, frame = cap.read()

# max_val,max_loc,templates[i].shape
first=frame.copy()
info = get_imgs(first)

# 设置初始化窗口
r, h, c, w = 250, 90, 400, 125
track_window = (c, r, w, h)
# 设置跟踪的ROI区域
roi = frame[r: r + h, c: c + w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask  = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
# 设置终止条件，迭代10次或移动1pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)


ret, frame = cap.read()
if ret:
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
    # 使用meanShift获得新位置
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    print('len pts:', len(pts), pts)
    img2 = cv2.polylines(frame, [pts], True,(255, 0, 0), 2)
    cv2.imshow('img2', img2)
    k = cv2.waitKey(int(1000/ fps))  # & 0xff
        
cv2.destroyAllWindows()
cap.release()

NameError: name 'get_imgs' is not defined

In [11]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
tmp1=cv2.imread("C:/Users/Luka/Desktop/585-hw2/template-flat.png")

In [26]:
temp_path = r'C:\Users\Luka\Desktop\Template\LAS\ribs'
for i in range(1,10):
    print(temp_path+"/"+"LAS"+"_"+str(i)+".png")

C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_1.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_2.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_3.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_4.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_5.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_6.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_7.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_8.png
C:\Users\Luka\Desktop\Template\LAS\ribs/LAS_9.png


In [28]:
cv2.imshow("w",cv2.imread(temp_path+"/"+"LAS"+"_"+str(9)+".png",0))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
def get_imgs(org,thr=None,opt=None):
    if not thr:
        thr = 0.6
    if not opt:
        opt = 1
    templates = []
    temp_path = r'C:\Users\Luka\Desktop\Template\LAS\ribs'
    for i in range(1,10):
        templates.append(cv2.imread(temp_path+"/"+"LAS"+"_"+str(i)+".png",0))####GRAYSCALE
    w, h = [template.shape[::-1][0] for template in templates], [template.shape[::-1][1] for template in templates]
    
    res_list=[]
    
    methods = ['cv2.TM_CCOEFF_NORMED']

    for meth in methods:
        method = eval(meth)
        img = org.copy()
        VL=[]
        for i in range(len(templates)):
            
            res = cv2.matchTemplate(img, templates[i], method)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            if max_loc[1] < org.shape[0]*0.2:
                VL.append([max_val,max_loc,i])
        VL.sort(key=lambda x:-x[0])

        for i in range(opt):
            if VL[i][0]>thr:
                top_left = VL[i][1]
                bottom_right = (top_left[0] + w[VL[i][2]], top_left[1] + h[VL[i][2]])
                cv2.rectangle(img, top_left, bottom_right, 255, 1)
                cv2.putText(img,"template"+str(VL[i][2]+1)+"Value="+str(VL[i][0])[:4],(top_left[0],top_left[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 1)
#             else:
#                 print("not draw")
    return VL[0],img


In [49]:
delay=int(1000/fps)

In [64]:
video.release()

In [3]:
templates = []
temp_path = r'C:\Users\Luka\Desktop\Template\LAS\ribs'
for i in range(1,10):
    templates.append(cv2.imread(temp_path+"/"+"LAS"+"_"+str(i)+".png",0))####GRAYSCALE
w, h = [template.shape[::-1][0] for template in templates], [template.shape[::-1][1] for template in templates]

In [4]:
video = cv2.VideoCapture('002_RI_full.mp4')

size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = video.get(cv2.CAP_PROP_FPS)
delay=int(1000/fps)

lb=0
rb=size[0]
depth=0
thr=0.9

success, frame = video.read()
info,img = get_imgs(frame[:,:,0],thr=thr)# max_val,max_loc,i
index=1

while(info[0]<thr and success):
    success, frame = video.read()
    index+=1
    if index%5==0:
        info,img = get_imgs(frame[:,:,0],thr=thr)
        
top_left,i=info[1],info[2]

roi=frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i]].copy()
print("ok")
# cv2.imshow("s",img)
# cv2.waitKey(0)

# ####get the template
# trackwindow=frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i],0].copy()
# cv2.imshow("s",trackwindow)
# cv2.waitKey(0)

# cv2.destroyAllWindows()
# video.release()

ok


In [6]:
rest=[]
while success:
    success,frame = video.read()
    index+=1
    if index%5==0:
        rest.append(frame.copy())

In [74]:
top_left,i=info[1],info[2]

roi=frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i]].copy()
print("ok")

TypeError: 'NoneType' object is not subscriptable

In [94]:
# trackwindow=frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i],0].copy()
cv2.imshow("tp",roi)
cv2.waitKey(0)
cv2.imshow("tp",cv2.resize(roi,(540,200)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [73]:
roi.shape

(134, 225)

In [85]:
cv2.destroyAllWindows()

In [8]:
track_window = (top_left[1],top_left[0],w[i],h[i])

# 从当前帧中框出一个小框
# roi = frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i]]
# RGB转为HSV更好处理
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi],[0],None,[255],[0,255])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# 设置迭代的终止标准，最多十次迭代
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

k=0
while(k<=len(rest)-1):
    frame = rest[k].copy()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 反向投影函数（特征提取函数）
    # 反向投影是一种记录给定图像中的像素点如何适应直方图模型像素分布的方式
    # 反向投影就是首先计算某一特征的直方图模型，然后使用模型去寻找图像中存在的特征
    # cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])

    # images:待处理的图像，图像格式为uint8或float32
    # channels:对应图像需要统计的通道，若是灰度图则为0，彩色图像B、G、R对应0、1、2
    # mask:掩膜图像。如果统计整幅图像就设置为None，否则这里传入设计的掩膜图像。
    # histSize表示这个直方图分成多少份（即多少个直方柱）
    # ranges:像素量化范围，通常为0 - 255。
    dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,255],1)

    # RotatedRect CamShift(InputArray probImage, Rect&window, TermCriteria criteria)。
    # probImage为输入图像直方图的反向投影图，
    # window为要跟踪目标的初始位置矩形框，
    # criteria为算法结束条件。
    # 函数返回一个有方向角度的矩阵。
    #
    ret, track_window = cv2.meanShift(dst, track_window, term_crit)

    # 在图片上绘制
    x, y, wp, hp = track_window
    img2 = cv2.rectangle(frame, (x, y), (x + wp, y + hp), 255, 2)
    cv2.imshow('img2', img2)
    cv2.waitKey(delay*5) & 0xff
    k+=1

#     cv2.waitKey(delay) & 0xff
            
cv2.destroyAllWindows()

In [87]:
cv2.destroyAllWindows()

In [48]:
trackwindow=frame[top_left[1]:top_left[1]+h[i],top_left[0]:top_left[0]+w[i],0].copy()
cv2.imshow("s",trackwindow)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [100]:
roi = cv2.imread("./gestures/1.jpg")

In [101]:
roi.shape

(158, 219, 3)

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

# ret判断是否读到图片
# frame读取到的当前帧的矩阵
# 返回的是元组类型，所以也可以加括号
ret, frame = cap.read()
# print(type(ret), ret)
# print(type(frame), frame)

# print(frame.shape)
# # 设置跟踪框参数
# r,h,c,w = 250,50,250,50  # simply hardcoded the values
# track_window = (c,r,w,h)

# # 从当前帧中框出一个小框
# roi = frame[r:r+h, c:c+w]
roi = cv2.imread("./gestures/1.jpg")
# RGB转为HSV更好处理
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# inRange函数设置亮度阈值
# 去除低亮度的像素点的影响
# eg. mask = cv2.inRange(hsv, lower_red, upper_red)

# 将低于和高于阈值的值设为0
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))

# 然后得到框中图像的直方图
# cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate ]])
# mask 即上文的阈值设置
# histSize表示这个直方图分成多少份（即多少个直方柱）
# range是表示直方图能表示像素值的范围
# 返回直方图
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[255],[0,255])

# 归一化函数cv2.normalize(src[, dst[, alpha[, beta[, norm_type[, dtype[, mask]]]]]])
# 返回dst类型
# 归一化就是要把需要处理的数据经过处理后（通过某种算法）限制在你需要的一定范围内
# src  - 输入数组
# dst  - 与src大小相同的输出数组
# alpha  - 范围值，   以便在范围归一化的情况下归一化到较低范围边界
# beta  - 范围归一化时的上限范围; 它不用于标准规范化
# normType  - 规范化类型 这里的NORM_MINMAX是数组的数值被平移或缩放到一个指定的范围，线性归一化。
# dtype  - 当为负数时，输出数组与src的类型相同；否则，它具有与src相同的通道数；深度=CV_MAT_DEPTH（dtype）
# mask  - 可选的操作掩码。
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# 设置迭代的终止标准，最多十次迭代
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # 反向投影函数（特征提取函数）
        # 反向投影是一种记录给定图像中的像素点如何适应直方图模型像素分布的方式
        # 反向投影就是首先计算某一特征的直方图模型，然后使用模型去寻找图像中存在的特征
        # cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])

        # images:待处理的图像，图像格式为uint8或float32
        # channels:对应图像需要统计的通道，若是灰度图则为0，彩色图像B、G、R对应0、1、2
        # mask:掩膜图像。如果统计整幅图像就设置为None，否则这里传入设计的掩膜图像。
        # histSize表示这个直方图分成多少份（即多少个直方柱）
        # ranges:像素量化范围，通常为0 - 255。
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # RotatedRect CamShift(InputArray probImage, Rect&window, TermCriteria criteria)。
        # probImage为输入图像直方图的反向投影图，
        # window为要跟踪目标的初始位置矩形框，
        # criteria为算法结束条件。
        # 函数返回一个有方向角度的矩阵。
        #
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Draw it on image
        pts = cv2.boxPoints(ret)

        # 类型转换int0()用于索引的整数(same as C ssize_t; normally either int32 or int64)
        pts = np.int0(pts)

        # 非填充多边形：polylines()
        # cv2.polylines(img, pts, isClosed, color[, thickness[, lineType[, shift]]])
        # img – 要画的图片
        # pts – 多边形的顶点
        # isClosed – 是否闭合线段
        # color – 颜色
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
        cv2.imshow('img2',img2)

        # 停止追踪按钮
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [42]:
cv2.destroyAllWindows()
cap.release()